## Winograd Schema Example

From - https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WSCollection.xml

This is a sample of how the engine can solve for specific cases of the Winograd Schema Challenge.

In [1]:
import os, sys
sys.path.insert(1, os.path.abspath('..\\..\\..'))

from thoughts.rules_engine import RulesEngine
import thoughts.unification
import pprint

engine = RulesEngine()

## Define the KB Rules

Define the knowledge base rules. These are a series of #when / #then rules that allow for wildcard matching. As rules and wilcards are matched in the #when portion, then #then portion is fired, which can cause additioanl rules to fire until no more matches occur.

For the Winograd Schema Challenge, the first rule will extract the premise from the incoming text assertion. Then the second rule will enhance the information that when an entity refuses another entity a permit, then the first entity may fear violence, and the second entity may advocate violence. Of course this is not always true, but may be true and so the engine generates it as a possibility for later evaluation.

Note that we wrap the first rule inside of a "parse" tag.  This is to prevent the second rule from matching against the initial incoming assertion. Think of the first rule as a rule that will handle the natural language parsing, and the second rule as the internal logic / knowledge base of facts about the world. There is no special signifigance of "parse" to the engine.

In [2]:
rules = [

    { "#when": {"parse": "?x because ?y"}, 
      "#then": ["?x"]},

    { "#when": "?x refused ?y a permit",
      "#then": ["?x feared violence",
               "?y advocated violence"]
    }
  ]

engine.load_rules_from_list(rules, "winograd-1")

## Assert the Premise

Define the premise and assert it. This will find all matching rules and continue to process matching rules, returning any conclusions that were genereated during the process.

In [3]:
premise = "The city councilmen refused the demonstrators a permit because they feared violence"
conclusions = engine.process({"parse": premise})
print("Conclusions:")
pprint.pprint(conclusions)

Conclusions:
['the city councilmen feared violence', 'the demonstrators advocated violence']


## Query the Conclusions

Now answer questions by querying the conclusions for a match. In this Winograd Schema Challenge example, the question is asked what the "it" refers to. It can be represented as a wildcard, ?x, and matched against the conclusions that were previous generated.

In [4]:
question = "?x feared violence"
answers = thoughts.unification.unify_item_with_list(question, conclusions)
print("Answer to '{0}':".format(question))
pprint.pprint(answers)

Answer to '?x feared violence':
[{'?x': 'the city councilmen'}]


## Another Query Example

Another example identified the other possible referent for "it".

In [5]:
question = "?x advocated violence"
answers = thoughts.unification.unify_item_with_list(question, conclusions)
print("Answer to '{0}':".format(question))
pprint.pprint(answers)

Answer to '?x advocated violence':
[{'?x': 'the demonstrators'}]
